In [1]:
import torch
import numpy as np
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import barabasi_albert_graph
from torch_geometric.transforms import LaplacianLambdaMax
from torch_geometric_temporal.nn.convolutional import TemporalConv, STConv, ASTGCN, MSTGCN, MTGNN, ChebConvAttention
from torch_geometric_temporal.nn.convolutional import GMAN, SpatioTemporalAttention, SpatioTemporalEmbedding

In [2]:
def create_mock_data(number_of_nodes, edge_per_node, in_channels):
    """
    Creating a mock feature matrix and edge index.
    """
    graph = nx.watts_strogatz_graph(number_of_nodes, edge_per_node, 0.5)
    edge_index = torch.LongTensor(np.array([edge for edge in graph.edges()]).T)
    X = torch.FloatTensor(np.random.uniform(-1, 1, (number_of_nodes, in_channels)))
    return X, edge_index

def create_mock_edge_weight(edge_index):
    """
    Creating a mock edge weight tensor.
    """
    return torch.FloatTensor(np.random.uniform(0, 1, (edge_index.shape[1])))

def create_mock_target(number_of_nodes, number_of_classes):
    """
    Creating a mock target vector.
    """
    return torch.LongTensor([np.random.randint(0, number_of_classes-1) for node in range(number_of_nodes)])

def create_mock_sequence(sequence_length, number_of_nodes, edge_per_node, in_channels, number_of_classes):
    """
    Creating mock sequence data
    
    Note that this is a static graph discrete signal type sequence
    The target is the "next" item in the sequence
    """
    input_sequence = torch.zeros(sequence_length, number_of_nodes, in_channels)
    
    X, edge_index = create_mock_data(number_of_nodes=number_of_nodes, edge_per_node=edge_per_node, in_channels=in_channels)
    edge_weight = create_mock_edge_weight(edge_index)
    targets = create_mock_target(number_of_nodes, number_of_classes)

    for t in range(sequence_length):
        input_sequence[t] = X+t

    return input_sequence, targets, edge_index, edge_weight

def create_mock_batch(batch_size, sequence_length, number_of_nodes, edge_per_node, in_channels, number_of_classes):
    """
    Creating a mock batch of sequences
    """
    batch = torch.zeros(batch_size, sequence_length, number_of_nodes, in_channels)
    batch_targets = torch.zeros(batch_size, number_of_nodes, dtype=torch.long)
    
    for b in range(batch_size):
        input_sequence, targets, edge_index, edge_weight = create_mock_sequence(sequence_length, number_of_nodes, edge_per_node, in_channels, number_of_classes)
        batch[b] = input_sequence
        batch_targets[b] = targets

    return batch, batch_targets, edge_index, edge_weight

In [3]:
"""
Testing MSTGCN block with changing edge index over time.
"""
node_count = 307
num_classes = 10
edge_per_node = 15

num_for_predict = 12
len_input = 12
nb_time_strides = 1

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

node_features = 2
nb_block = 2
K = 3
nb_chev_filter = 64
nb_time_filter = 64
batch_size = 1

model = MSTGCN( nb_block, node_features, K, nb_chev_filter,
                nb_time_filter, nb_time_strides, 
                num_for_predict, len_input ).to(device)
T = len_input
x_seq = torch.zeros([batch_size,node_count, node_features,T]).to(device)
target_seq = torch.zeros([batch_size,node_count,T]).to(device)
edge_index_seq = []

for b in range(batch_size):
    for t in range(T):
        x, edge_index = create_mock_data(node_count, 2*(t+1), node_features)
        #print(x.shape, edge_index.shape)
        x_seq[b,:,:,t] = x.to(device)
        #if b == 0:
        edge_index_seq.append( edge_index.to(device) )
        target = create_mock_target(node_count, num_classes).to(device)
        target_seq[b,:,t] = target

shuffle = True
#print(x_seq.shape, target_seq.shape, len(edge_index_seq) )
train_dataset = torch.utils.data.TensorDataset( x_seq, target_seq )
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

for batch_data in train_loader:
    encoder_inputs, _ = batch_data
    outputs1 = model(encoder_inputs, edge_index_seq)
    outputs2 = model(encoder_inputs, edge_index_seq[0])

assert outputs1.shape == (batch_size, node_count, num_for_predict)
assert outputs2.shape == (batch_size, node_count, num_for_predict)


In [4]:
print(x_seq.shape, target_seq.shape, len(edge_index_seq) )

torch.Size([1, 307, 2, 12]) torch.Size([1, 307, 12]) 12


In [5]:
x_seq.dtype, target_seq.dtype, edge_index_seq[0].dtype

(torch.float32, torch.float32, torch.int64)

In [6]:
model( x_seq, edge_index_seq )

tensor([[[-0.8175,  0.6724,  1.0434,  ..., -0.9552,  0.8803,  0.2471],
         [-0.8206,  0.2323,  0.9486,  ..., -0.9708,  0.7109,  0.4421],
         [-0.8519,  0.3284,  0.9682,  ..., -0.6460,  0.9835,  0.1755],
         ...,
         [-1.1424,  0.5672,  1.1176,  ..., -1.2706,  0.5072,  0.9132],
         [-0.7091,  0.6109,  1.1609,  ..., -1.4413,  0.9390,  0.6822],
         [-0.9816,  0.7417,  1.0512,  ..., -1.3903,  0.6515,  0.8488]]],
       grad_fn=<PermuteBackward>)

In [7]:
model( x_seq, edge_index_seq[0] )

tensor([[[-0.9215,  0.4003,  1.0774,  ..., -0.6848,  0.9072,  0.3584],
         [-0.9199,  0.1810,  1.1363,  ..., -0.9495,  0.8687,  0.4297],
         [-0.8190,  0.2504,  1.0142,  ..., -0.5723,  0.8829,  0.3445],
         ...,
         [-0.8948,  0.4124,  0.5023,  ..., -0.7351,  0.7836, -0.1730],
         [-0.7876,  0.4367,  0.6315,  ..., -1.2420,  0.8399,  0.3512],
         [-0.7813,  0.5204,  0.4176,  ..., -0.9701,  0.7676,  0.3758]]],
       grad_fn=<PermuteBackward>)

In [8]:
edge_index_seq[0].dtype

torch.int64

In [9]:
x_seq[0].shape, len( edge_index_seq ), edge_index_seq[0].shape

(torch.Size([307, 2, 12]), 12, torch.Size([2, 307]))

In [10]:
x_seq[0][0], edge_index_seq

(tensor([[ 0.9647, -0.9167,  0.7907, -0.0637, -0.8508, -0.9583, -0.7610, -0.8589,
          -0.2375,  0.2733, -0.5891, -0.7227],
         [-0.9545, -0.6329, -0.4435, -0.4049,  0.5680,  0.8029,  0.7929,  0.3561,
           0.2712, -0.6077, -0.8194, -0.1498]]),
 [tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   7,   8,   9,  10,  11,  12,
            13,  13,  14,  14,  14,  15,  15,  16,  17,  18,  19,  20,  21,  22,
            22,  23,  23,  24,  24,  25,  26,  27,  27,  27,  27,  28,  28,  29,
            30,  31,  32,  32,  33,  34,  34,  35,  35,  36,  36,  37,  38,  38,
            39,  40,  41,  41,  42,  43,  44,  45,  45,  46,  47,  48,  48,  48,
            49,  49,  50,  51,  52,  53,  53,  53,  54,  54,  55,  55,  56,  57,
            58,  60,  61,  62,  62,  63,  64,  65,  65,  66,  66,  68,  69,  69,
            70,  70,  71,  72,  74,  75,  75,  76,  76,  77,  77,  78,  79,  80,
            81,  81,  83,  84,  84,  85,  86,  87,  88,  89,  90,  91,  91,  91,
           

In [11]:
edge_index_seq[0].dtype

torch.int64

In [12]:
x_seq.shape, edge_index_seq[0].shape, edge_index_seq[2].shape

(torch.Size([1, 307, 2, 12]), torch.Size([2, 307]), torch.Size([2, 921]))

In [13]:
model( x_seq, edge_index_seq )

tensor([[[-0.8175,  0.6724,  1.0434,  ..., -0.9552,  0.8802,  0.2471],
         [-0.8206,  0.2324,  0.9486,  ..., -0.9708,  0.7110,  0.4421],
         [-0.8519,  0.3284,  0.9682,  ..., -0.6460,  0.9835,  0.1755],
         ...,
         [-1.1424,  0.5672,  1.1176,  ..., -1.2706,  0.5072,  0.9132],
         [-0.7091,  0.6109,  1.1609,  ..., -1.4413,  0.9390,  0.6822],
         [-0.9816,  0.7417,  1.0512,  ..., -1.3903,  0.6515,  0.8488]]],
       grad_fn=<PermuteBackward>)

In [14]:
x_seq.shape, edge_index_seq[0].shape

(torch.Size([1, 307, 2, 12]), torch.Size([2, 307]))

In [15]:
torch.max( model( x_seq, edge_index_seq )-model( x_seq, edge_index_seq[0] ) )

tensor(1.0862, grad_fn=<MaxBackward1>)

In [16]:
[ i for i in edge_index_seq ]

[tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   7,   8,   9,  10,  11,  12,
           13,  13,  14,  14,  14,  15,  15,  16,  17,  18,  19,  20,  21,  22,
           22,  23,  23,  24,  24,  25,  26,  27,  27,  27,  27,  28,  28,  29,
           30,  31,  32,  32,  33,  34,  34,  35,  35,  36,  36,  37,  38,  38,
           39,  40,  41,  41,  42,  43,  44,  45,  45,  46,  47,  48,  48,  48,
           49,  49,  50,  51,  52,  53,  53,  53,  54,  54,  55,  55,  56,  57,
           58,  60,  61,  62,  62,  63,  64,  65,  65,  66,  66,  68,  69,  69,
           70,  70,  71,  72,  74,  75,  75,  76,  76,  77,  77,  78,  79,  80,
           81,  81,  83,  84,  84,  85,  86,  87,  88,  89,  90,  91,  91,  91,
           93,  93,  94,  95,  95,  96,  99, 100, 101, 102, 105, 105, 106, 106,
          107, 108, 108, 109, 110, 111, 113, 113, 114, 115, 116, 116, 118, 119,
          120, 120, 121, 121, 121, 122, 123, 123, 124, 125, 127, 127, 128, 129,
          129, 130, 131, 131, 132, 132, 

### Load cellular data

In [17]:
import os

In [18]:
"""
Testing MSTGCN block with changing edge index over time.
"""
nb_time_strides = 1

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_for_predict = 4
len_input = 12
node_features = 3

nb_block = 1
K = 2
nb_chev_filter = 2
nb_time_filter = 2
batch_size = 1

model = MSTGCN( nb_block, node_features, K, nb_chev_filter,
                nb_time_filter, nb_time_strides, 
                num_for_predict, len_input ).to(device)
T = len_input

node_count = 14585

num_for_predict = 1
len_input = 12
nb_time_strides = 1

node_features = 3
nb_block = 2
x_seq = torch.zeros([batch_size,node_count, node_features,T]).to(device)
target_seq = torch.zeros([batch_size,node_count,T]).to(device)
edge_index_seq = []

source = '/media/storage_3/abiricz/Mobilcell/TimeIntervalGraphs/'
files = np.array( sorted([ i for i in os.listdir(source) ]) )[:12]

for b in range(batch_size):
    for t in range(T):
        loaded = np.load( source+files[t] )
        x = torch.LongTensor( loaded['data_x'] )
        edge_index = torch.LongTensor( loaded['data_edge_index']  )
        x_seq[b,:,:,t] = x.to(device)
        #if b == 0:
        edge_index_seq.append( edge_index.to(device) )
        #target = create_mock_target(node_count, num_classes).to(device)
        #target_seq[b,:,t] = target

In [19]:
x_seq.shape, len(edge_index_seq), edge_index_seq[0].shape

(torch.Size([1, 14585, 3, 12]), 12, torch.Size([2, 28863]))

In [22]:
model( x_seq, edge_index_seq )

tensor([[[0.0630, 1.0753, 0.3691, 1.3912],
         [0.0630, 1.0753, 0.3691, 1.3912],
         [0.0630, 1.0753, 0.3691, 1.3912],
         ...,
         [0.0630, 1.0753, 0.3691, 1.3912],
         [0.0630, 1.0753, 0.3691, 1.3912],
         [0.0630, 1.0753, 0.3691, 1.3912]]], grad_fn=<PermuteBackward>)